## Improvements of measure_PPI.py
Create 2025-01-07

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import time

In [2]:
libpath = Path("../andreas lib").resolve()
print(libpath)
sys.path.insert(0, str(libpath))
import measure_PPI

D:\Eigene Datein\Programmieren\Git\abrilka\bachelorthesis\andreas lib


In [3]:
structure_basePath = Path("../ressources/ISS AF_DMI_structures").resolve()
structure_folders = [structure_basePath / p for p in ['AF_DMI_structures1', 'AF_DMI_structures2', 'AF_DMI_structures3']]
solved_basePath = Path("../ressources/ISS DMI_solved_structures").resolve()
solvedHydrogen_basePath = Path("../ressources/ISS DMI_solved_structures hydrogens").resolve()

In [4]:
#Loading sample structure
sampleStructure_name = "LIG_NRP_CendR_1_2ORZ"
sampleStructure_path = structure_folders[1] / sampleStructure_name / "ranked_0.pdb"
#sampleStructure_name = "DEG_MDM2_SWIB_1_1YCR"
#sampleStructure_path = structure_folders[0] / sampleStructure_name / "ranked_1.pdb"
measure_PPI.logger.setLevel(measure_PPI.logging.DEBUG)
structure_biopy, atomarray_biotite = measure_PPI.OpenStructure(sampleStructure_path, sampleStructure_name)
measure_PPI.EvaluateStructure(sampleStructure_path, sampleStructure_name)

[2025-01-07 14:41:41,572 | measure_PPI | DEBUG] Runtime reading structure LIG_NRP_CendR_1_2ORZ (file ranked_0.pdb): 26.8ms
[2025-01-07 14:41:41,594 | measure_PPI | DEBUG] Runtime reading structure LIG_NRP_CendR_1_2ORZ (file ranked_0.pdb): 21.3ms
[2025-01-07 14:41:41,672 | measure_PPI | DEBUG] Sasa values: Chain 1 = 7380.857, Chain 2 = 734.776, Total = 7417.026
[2025-01-07 14:41:41,673 | measure_PPI | DEBUG] Runtime calculate_buried_area: 78.2ms (2.0ms model buiilding, 20.7ms loading, 55.5ms sasa calc)
[2025-01-07 14:41:41,774 | measure_PPI | DEBUG] Runtime calculate_hbonds: 100.5ms (0.4ms generating chains, 97.6ms bond list, 2.5ms hbonds)
[2025-01-07 14:41:41,784 | measure_PPI | DEBUG] Runtime calculate_min_distance: 9.8ms (0.7ms generating chains, 9.1ms calculating distance)
[2025-01-07 14:41:41,786 | measure_PPI | DEBUG] Runtime calculate_saltbridges: 0.8ms
[2025-01-07 14:41:41,791 | measure_PPI | DEBUG] Runtime calculate_hydrophobic_interactions: 5.2ms
[2025-01-07 14:41:41,791 | mea

{'structure_name': 'LIG_NRP_CendR_1_2ORZ',
 'file': 'ranked_0.pdb',
 'hbonds': 5,
 'salt_bridges': 2,
 'buried_area': 698.607,
 'min_distance': 6.297,
 'hydrophobic_interactions': 0}

Dev code

In [18]:
import biotite.structure as struc
import biotite.structure.io.pdb as bt_pdb
from Bio.PDB import PDBParser
from Bio.PDB.Structure import Structure as BioPy_PDBStructure
from Bio.PDB.Model import Model as BioPy_PDBModel
from Bio.PDB.PDBExceptions import PDBConstructionException
import math

In [ ]:
cutoff=5.0

# If True, the algorithm will not return NaN if only the backbone is above cutoff but some atoms of the
# residues are below cutoff distance
atom_cutoff = False 

ti = time.perf_counter()
chains = struc.get_chains(atomarray_biotite)
assert len(chains) == 2

chain1 = atomarray_biotite[atomarray_biotite.chain_id == chains[0]]
chain2 = atomarray_biotite[atomarray_biotite.chain_id == chains[1]]
t1 = time.perf_counter()

# For atom_cutoff = True the previous behaviour
# To reduce computational cost, reversed the order from first filtering that at least one atom of a residue has a
# atom of the opposite chain below cutoff distance and then filtering for CA. 
# 

chain1_backbone = chain1[chain1.atom_name == "CA"]
chain2_backbone = chain2[chain2.atom_name == "CA"]
min_distance = float("inf")

for ca1 in chain1_backbone:
    for ca2 in chain2_backbone:
        # Syntactic sugar: Use := operator to calculate dist only one
        if (dist := struc.distance(ca1, ca2)) < cutoff:
            print(f"Min s distance for {ca1.res_id} and {ca2.res_id} is {round(float(dist), 3)}")
            min_distance = min(min_distance, dist)
            continue
        elif dist <= 2*cutoff and atom_cutoff: 
            # To save computational cost, use also a cutoff value for backbone estimated to be twice the usual cutoff
            for a1 in chain1[chain1.res_id == ca1.res_id]:
                for a2 in chain2[chain2.res_id == ca2.res_id]:
                    if struc.distance(a1, a2) <= cutoff:
                        break
                else: # Runs after look finished normally
                    continue
                break # This only runs if there is a break in the inner loop because of previous continue statement
            else:
                break
                # Only calculate min_distance if there is the atom wise distance of the residues is below cutoff
            print(f"Min distance for {ca1.res_id} and {ca2.res_id} is {round(float(dist), 3)}")
            min_distance = min(min_distance, dist)
tf = time.perf_counter()
print(f"{(round(1000*(tf-ti), 1))} ms")
print(min_distance)

In [ ]:
cutoff=5.0 

# If True, the algorithm will not return NaN if only the backbone is above cutoff but some atoms of the
# residues are below cutoff distance
atom_cutoff = False 

ti = time.perf_counter()
chains = struc.get_chains(atomarray_biotite)
assert len(chains) == 2

chain1 = atomarray_biotite[atomarray_biotite.chain_id == chains[0]]
chain2 = atomarray_biotite[atomarray_biotite.chain_id == chains[1]]
t1 = time.perf_counter()

# To reduce computational cost, removed constrained to interface atoms only as this does not affect the result.
# The previous code used for this first pymol selection algebra with the command 'within' which filters
# by cutoff distance atom wise before filtering for CA atoms. Now the code filters first for CA and then for
# cutoff distance --> less atoms to compare.

chain1_backbone = chain1[chain1.atom_name == "CA"]
chain2_backbone = chain2[chain2.atom_name == "CA"]
min_distance = float("inf")
for a1 in chain1_backbone:
    for a2 in chain2_backbone:
        min_distance = min(min_distance, struc.distance(a1, a2))
tf = time.perf_counter()
print(f"{(round(1000*(tf-ti), 1))} ms")
print(min_distance)

In [ ]:
chain1_atoms = chain1[chain1.res_id in chain1_interface_atoms]
print(chain1_atoms)

In [ ]:
measure_PPI.calculate_min_distance(atomarray_biotite, max_cutoff=200)

In [ ]:
cutoff = 5.0
max_cutoff = 10

ti = time.perf_counter()
chains = struc.get_chains(atomarray_biotite)
assert len(chains) == 2

chain1 = atomarray_biotite[atomarray_biotite.chain_id == chains[0]]
chain2 = atomarray_biotite[atomarray_biotite.chain_id == chains[1]]

chain1_backbone = chain1[chain1.atom_name == "CA"]
chain2_backbone = chain2[chain2.atom_name == "CA"]

min_distance = float("inf")
t1 = time.perf_counter()

# max_cutoff is implemented to mimic the same behaviour as the ISS code which used pymol.

for ca1 in chain1_backbone:
    for ca2 in chain2_backbone:
        if (dist := struc.distance(ca1, ca2)) < cutoff:
            min_distance = min(min_distance, dist)
            continue
        elif dist <= max_cutoff and dist < min_distance: # If max_cutoff is set, check the individual atoms
            for a1 in chain1[chain1.res_id == ca1.res_id]:
                for a2 in chain2[chain2.res_id == ca2.res_id]:
                    if struc.distance(a1, a2) <= cutoff:
                        break
                else: # Runs after loop finished normally
                    continue
                break # This only runs if there is a break in the inner loop because of previous continue statement
            else:
                # Only calculate min_distance if there is the atom wise distance of the residues is below cutoff
                continue
            min_distance = min(min_distance, dist)

tf = time.perf_counter()

d = round(min_distance, 3) if math.isfinite(min_distance) else float('NaN')
print(d)